In [2]:
%matplotlib inline

import csv
import itertools
import os
import pprint
from xml.etree import ElementTree

import h5py
import numpy as np
import pandas as pd
import skimage.io

import matplotlib.pyplot as plt

In [3]:
base_dir = '/data/slmStimPipeline'

recording = 0

if recording == 0:
    session_name = '20200124M74'
    recording_name = 'stimL23-000'
    recording_name2 = 'stim3dL23withBeam-001'
elif recording == 1:
    session_name = '20200201M79'
    recording_name = 'VRmm-000'
    recording_name2 = recording_name
elif recording == 2:
    session_name = '20200202M79'
    recording_name = 'stimL23-000'
    recording_name2 = 'stimL23-001'
else:
    raise ValueError('Bad recording %s' % recording)

base = os.path.join(base_dir, session_name, recording_name, recording_name2)
print(base)

/data/slmStimPipeline/20200124M74/stimL23-000/stim3dL23withBeam-001


In [4]:
def metadata(base):
    mdata_root = ElementTree.fromstring(open(base + '.xml').read())

    def state_value(key, type_fn=str):
        element = mdata_root.find(f'.//PVStateValue[@key="{key}"]')
        value = element.attrib['value']
        return type_fn(value) 

    def indexed_value(key, index, type_fn=None):
        element = mdata_root.find(f'.//PVStateValue[@key="{key}"]/IndexedValue[@index="{index}"]')
        value = element.attrib['value']
        return type_fn(value)

    num_frames = len(mdata_root.findall('Sequence'))
    num_channels = len(mdata_root.find('Sequence/Frame').findall('File'))
    num_z_planes = len(mdata_root.find('Sequence').findall('Frame'))
    num_y_px = state_value('linesPerFrame', int)
    num_x_px = state_value('pixelsPerLine', int)

    laser_power = indexed_value('laserPower', 0, float)
    laser_wavelength = indexed_value('laserWavelength', 0, int)
    
    frame_period = state_value('framePeriod', float)
    optical_zoom = state_value('opticalZoom', float)

    fname_voltage_xml = base + '_Cycle00001_VoltageRecording_001.xml'
    voltage_root = ElementTree.fromstring(open(fname_voltage_xml).read()) 

    channels = {}
    for signal in voltage_root.findall('Experiment/SignalList/VRecSignal'):
        channel = int(signal.find('Channel').text)
        name = signal.find('Name').text
        channels[name] = channel

    return {
        'size': {'frames': num_frames, 
                 'channels': num_channels, 
                 'z_planes': num_z_planes,
                 'y_px': num_y_px,
                 'x_px': num_x_px},
        'laser': {'power': laser_power, 'wavelength': laser_wavelength},
        'period': frame_period,
        'optical_zoom': optical_zoom,
        'channels': channels,
    }

md_dct = metadata(base)
pprint.pprint(md_dct)

{'channels': {'AO0': 5,
              'FieldStimulator': 7,
              'OptogeneticPockels': 6,
              'VR timestamps': 4,
              'frame starts': 0,
              'secondary': 1,
              'uncagingX': 2,
              'uncagingY': 3},
 'laser': {'power': 160.234489496825, 'wavelength': 680},
 'optical_zoom': 1.4,
 'period': 0.033069004,
 'size': {'channels': 2,
          'frames': 4465,
          'x_px': 512,
          'y_px': 512,
          'z_planes': 4}}


In [5]:
fname_voltage_csv = base + '_Cycle00001_VoltageRecording_001.csv'
df = pd.read_csv(fname_voltage_csv, index_col='Time(ms)', skipinitialspace=True)

/opt/conda/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:
size = md_dct['size']
shape = (size['frames'], size['z_planes'])
y_px = size['y_px']

buffer = 5
shift = 3
chn = 3  # Get this empirically

frame = df['frame starts']
frame_start = frame[frame.diff() > 2.5].index

stim = df['FieldStimulator']
stim_start = stim[stim.diff() > 2.5].index
stim_stop = stim[stim.diff() < -2.5].index

In [7]:
def get_loc(times):
    interp = np.interp(times, frame_start, range(len(frame_start)))
    indices = interp.astype(np.int)
    y_offset = y_px * (interp - indices)
    return np.transpose(np.unravel_index(indices, shape)), y_offset

ix_start, y_off_start = get_loc(stim_start)
ix_stop, y_off_stop = get_loc(stim_stop)

In [11]:
index = 0
frame = []
z_plane = []
y_px_start = []
y_px_stop = []
for (ix_start_cyc, ix_start_z), (ix_stop_cyc, ix_stop_z), y_min, y_max in zip(ix_start, ix_stop, y_off_start, y_off_stop):
    if (ix_start_cyc == ix_stop_cyc) and (ix_start_z == ix_stop_z):
        frame.append(ix_start_cyc)
        z_plane.append(ix_start_z)
        y_px_start.append(y_min)
        y_px_stop.append(y_max)
    else:
        frame.append(ix_start_cyc)
        z_plane.append(ix_start_z)
        y_px_start.append(y_min)        
        y_px_stop.append(y_px)
        
        frame.append(ix_stop_cyc)
        z_plane.append(ix_stop_z)
        y_px_start.append(0)
        y_px_stop.append(y_max)
        
df = pd.DataFrame({'frame': frame, 
                   'z_plane': z_plane, 
                   'y_min': y_px_start, 
                   'y_max': y_px_stop})
os.makedirs('/data/slm_test', exist_ok=True)
df.to_hdf('/data/slm_test/artefact.h5', 'data')

In [12]:
%%time

def read(frame, chn, z):
    fname = base + f'_Cycle{frame+1:05d}_Ch{chn}_{z+1:06d}.ome.tif'
    return skimage.io.imread(fname)

shape = (size['frames'], size['z_planes'], size['y_px'], size['x_px'])
dtype = read(0, chn, 0).dtype
print((shape, dtype))
data = np.zeros(shape, dtype)

for frame in range(size['frames']):
    for z_plane in range(size['z_planes']):
        data[frame, z_plane] = read(frame, chn, z_plane)       

/opt/conda/lib/python3.7/site-packages/skimage/external/tifffile/tifffile.py:3766: RuntimeWarning: invalid value encountered in true_divide
  'z_distance': values[:, 0] / values[:, 1],
/opt/conda/lib/python3.7/site-packages/skimage/external/tifffile/tifffile.py:1759: UserWarning: ome-xml: not an ome-tiff master file
  warnings.warn("ome-xml: not an ome-tiff master file")


((4465, 4, 512, 512), dtype('uint16'))
CPU times: user 13.4 s, sys: 4.5 s, total: 17.9 s
Wall time: 17.9 s


In [13]:
%%time
with h5py.File('/data/slm_test/orig.h5', 'w') as f_orig:
    f_orig.create_dataset('data', data=data)

CPU times: user 36.6 ms, sys: 4.68 s, total: 4.71 s
Wall time: 4.65 s


In [14]:
for row in df.itertuples():
    y_slice = slice(int(row.y_min) + shift,
                    int(row.y_max) + shift + buffer + 1)
    before = data[row.frame - 1, row.z_plane, y_slice]
    after = data[row.frame + 1, row.z_plane, y_slice]
    data[row.frame, row.z_plane, y_slice] = (before + after) / 2

In [16]:
%%time
with h5py.File('/data/slm_test/corrected.h5', 'w') as f_corr:
    f_corr.create_dataset('data', data=data)

CPU times: user 20.4 ms, sys: 4.65 s, total: 4.67 s
Wall time: 4.61 s


In [17]:
from suite2p import run_s2p
ops = run_s2p.default_ops()

In [32]:
db = {
    'h5py': '/data/slm_test/corrected.h5',
    'nplanes': size['z_planes'],
    'fast_disk': '/data/slm_test',
    'data_path': [],
}

In [33]:
opsEnd=run_s2p.run_s2p(ops=ops,db=db)

{'h5py': '/data/slm_test/corrected.h5', 'nplanes': 4, 'fast_disk': '/data/slm_test', 'data_path': []}
h5
time 34.68 sec. Wrote h5py to binaries for 4 planes
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
----------- REGISTRATION
registering 4465 frames
Reference frame, 4.95 sec.
2500/4465 frames, 37.58 sec.
4465/4465 frames, 67.38 sec.
bad frames file path: /data/slm_test/bad_frames.npy
----------- Total 72.59 sec
Registration metrics, 17.17 sec.
----------- ROI DETECTION AND EXTRACTION
Binning movie in chunks of length 10
Binned movie [446,509,509], 2.37 sec.
NOTE: estimated spatial scale ~6 pixels, time epochs 1.00, threshold 5.00 
0 ROIs, score=129.17
1000 ROIs, score=13.08
2000 ROIs, score=11.14
3000 ROIs, score=9.84
4000 ROIs, score=8.87
Found 5000 ROIs, 30.53 sec
NOTE: applying classifier /opt/conda/lib/python3.7/site-packages/suite2p/extraction/../classifiers/classifier_user.npy
After removing overlaps, 4856 ROIs remain
Masks made in 123.78 sec.
Extracted fluorescence from

In [19]:
ops_orig = np.load('/data/slmStimPipeline/20200124M74/stimL23-000/suite2p/ops1.npy', allow_pickle=True)

In [34]:
ops_orig[0].keys()

dict_keys(['look_one_level_down', 'fast_disk', 'delete_bin', 'mesoscan', 'bruker', 'h5py', 'h5py_key', 'save_path0', 'save_folder', 'subfolders', 'nplanes', 'nchannels', 'functional_chan', 'tau', 'fs', 'force_sktiff', 'frames_include', 'preclassify', 'save_mat', 'combined', 'aspect', 'do_bidiphase', 'bidiphase', 'bidi_corrected', 'do_registration', 'two_step_registration', 'keep_movie_raw', 'nimg_init', 'batch_size', 'maxregshift', 'align_by_chan', 'reg_tif', 'reg_tif_chan2', 'subpixel', 'smooth_sigma_time', 'smooth_sigma', 'th_badframes', 'pad_fft', 'nonrigid', 'block_size', 'snr_thresh', 'maxregshiftNR', '1Preg', 'spatial_hp', 'pre_smooth', 'spatial_taper', 'roidetect', 'sparse_mode', 'diameter', 'spatial_scale', 'connected', 'nbinned', 'max_iterations', 'threshold_scaling', 'max_overlap', 'high_pass', 'inner_neuropil_radius', 'min_neuropil_pixels', 'allow_overlap', 'chan2_thres', 'baseline', 'win_baseline', 'sig_baseline', 'prctile_baseline', 'neucoeff', 'xrange', 'yrange', 'data_pa

In [40]:
for key in ops_orig[0]:
    if key not in {'filelist', 'yblock', 'xblock', 'NRsm', 'refImg', 'meanImg', 'regPC'}:
        print(key, ops_orig[0][key])

look_one_level_down 0.0
fast_disk C:/Users/drinnenb/Data/suite2p_Results/20200124M79new/stim3Dl23withbeam\suite2p\plane0
delete_bin False
mesoscan False
bruker False
h5py []
h5py_key data
save_path0 C:/Users/drinnenb/Data/suite2p_Results/20200124M79new/stim3Dl23withbeam
save_folder []
subfolders []
nplanes 4
nchannels 1
functional_chan 1
tau 1.0
fs 7.45
force_sktiff False
frames_include -1
preclassify 0.0
save_mat True
combined True
aspect 1.0
do_bidiphase False
bidiphase 0.0
bidi_corrected True
do_registration 1
two_step_registration 0.0
keep_movie_raw False
nimg_init 300
batch_size 500
maxregshift 0.1
align_by_chan 1
reg_tif False
reg_tif_chan2 False
subpixel 10
smooth_sigma_time 0.0
smooth_sigma 1.15
th_badframes 1.0
pad_fft False
nonrigid True
block_size [128, 128]
snr_thresh 1.2
maxregshiftNR 5.0
1Preg False
spatial_hp 50.0
pre_smooth 2.0
spatial_taper 50.0
roidetect True
sparse_mode False
diameter [12 12]
spatial_scale 0
connected True
nbinned 637
max_iterations 20
threshold_scal